In [1]:
import math
import numpy as np

In [ ]:
def theta_from_beta(beta_rad: float, M: float, gamma: float = 1.4) -> float:
    """Return theta (rad) given beta (rad) and Mach M (perfect gas)."""
    term1 = 2.0 / math.tan(beta_rad)
    term2 = M**2 * math.sin(beta_rad)**2 - 1.0
    term3 = M**2 * (gamma + math.cos(2.0 * beta_rad)) + 2.0
    return math.atan(term1 * term2 / term3)

def solve_beta_bisection(theta_deg: float,
                         M: float,
                         gamma: float = 1.4,
                         tol: float = 1.0e-10,
                         max_iter: int = 100) -> float:
    """
    Solve for the shock angle beta (deg) by bisection, given
    the flow deflection theta (deg) and upstream Mach M.
    Returns the weak-shock solution.
    """
    theta_rad = math.radians(theta_deg)

    # Lower bound: just above the Mach line
    beta_min = math.asin(1.0 / M) + 1.0e-6

    def f(beta):
        return theta_from_beta(beta, M, gamma) - theta_rad

    f_prev = f(beta_min)
    beta_prev = beta_min
    beta_low = beta_high = None

    # --- 1. automatic bracketing -----------------------------------------
    # Scan beta from beta_min up to 89.9 deg until f changes sign.
    for beta_deg_scan in np.linspace(math.degrees(beta_min) + 0.05, 89.9, 2000):
        beta = math.radians(beta_deg_scan)
        f_curr = f(beta)
        if f_prev * f_curr <= 0.0:          # sign change -> root inside
            beta_low, beta_high = beta_prev, beta
            break
        beta_prev, f_prev = beta, f_curr

    if beta_low is None:
        raise ValueError(
            "theta exceeds the maximum deflection (theta_max) for this Mach; "
            "no attached oblique shock is possible."
        )

    # --- 2. classical bisection ------------------------------------------
    for _ in range(max_iter):
        beta_mid = 0.5 * (beta_low + beta_high)
        f_mid = f(beta_mid)
        if abs(f_mid) < tol:
            return math.degrees(beta_mid)
        if f(beta_low) * f_mid < 0.0:
            beta_high = beta_mid
        else:
            beta_low = beta_mid

    raise RuntimeError("Bisection did not converge; adjust tol or max_iter.")


In [3]:
for theta in (0.5, 5.0, 10.0):          # a few test cases
	M = 2.0
	beta = solve_beta_bisection(theta, M)
	print(f"theta = {theta:>4.1f} deg,  M = {M}  ->  beta = {beta:8.4f} deg")


theta =  0.5 deg,  M = 2.0  ->  beta =  30.4028 deg
theta =  5.0 deg,  M = 2.0  ->  beta =  34.3016 deg
theta = 10.0 deg,  M = 2.0  ->  beta =  39.3139 deg
